diff?

diff code blocks?


TODO: make compatible with PECL highlight and PHP code


In [ ]:
// npm install diff --save
var {JSDOM} = require('jsdom');
var JsDiff = require('diff');
const Prism = require('prismjs');
require('prismjs/components/prism-javascript');
require('prismjs/plugins/keep-markup/prism-keep-markup.js'); // Make sure it's installed

function htmlEntities(str) {
    return str.replace(/[\u00A0-\u9999<>\&]/gim, i => '&#' + i.charCodeAt(0) + ';');
}

function prismHighlightHtml(code, lang = 'javascript') {
    // Load the language if not already loaded
    if (!Prism.languages[lang]) {
        try {
            require(`prismjs/components/prism-${lang}.js`);
        } catch (err) {
            throw new Error(`Prism language "${lang}" could not be loaded.`);
        }
    }

    const dom = new JSDOM(`<body><pre><code class="language-${lang}">${htmlEntities(code)}</code></pre></body>`, {
        contentType: 'text/html',
    });

    // Required to mock the DOM API Prism plugins expect
    dom.window.document.createRange = () => ({
        createContextualFragment: str => JSDOM.fragment(str),
    });

    const codeElement = dom.window.document.querySelector('code');
    Prism.highlightElement(codeElement); // Triggers plugin processing

    return codeElement.innerHTML;
}

function diffTwoTexts(left, right) {
    var diff = JsDiff.diffWords(left, right);
    var code = '';
    for (var i = 0; i < diff.length; i++) {
        if (diff[i].added && diff[i + 1] && diff[i + 1].removed) {
            var swap = diff[i];
            diff[i] = diff[i + 1];
            diff[i + 1] = swap;
        }

        if (diff[i].removed) {
            code += '<span class="del">' + htmlEntities(diff[i].value) + '</span>';
        } else if (diff[i].added) {
            code += '<span class="ins">' + htmlEntities(diff[i].value) + '</span>';
        } else {
            code += '<span>' + htmlEntities(diff[i].value) + '</span>';
        }
    }
    return '<style>' + importer.interpret('prism-style.css').code + '</style>\n<pre>' + prismHighlightHtml(code) + `</pre>`;
}
module.exports = diffTwoTexts;
diffTwoTexts;



prism-style.css?

ROUTE = /prism-style.css


In [ ]:

.token.comment,
.token.prolog,
.token.doctype,
.token.cdata {
    color: slategray;
}

.token.punctuation {
    color: #999;
}

.namespace {
    opacity: .7;
}

.token.property,
.token.tag,
.token.boolean,
.token.number,
.token.constant,
.token.symbol,
.token.deleted {
    color: #905;
}

.token.selector,
.token.attr-name,
.token.string,
.token.char,
.token.builtin,
.token.inserted {
    color: #690;
}

.token.operator,
.token.entity,
.token.url,
.language-css .token.string,
.style .token.string {
    color: #a67f59;
    background: hsla(0, 0%, 100%, .5);
}

.token.atrule,
.token.attr-value,
.token.keyword {
    color: #07a;
}

.token.function {
    color: #DD4A68;
}

.token.regex,
.token.important,
.token.variable {
    color: #e90;
}

.token.important,
.token.bold {
    font-weight: bold;
}

.token.italic {
    font-style: italic;
}

.token.entity {
    cursor: help;
}
.del {
    text-decoration: none;
    background: #fadad7;
}
.ins {
    background: #eaf2c2;
    text-decoration: none;
}



test prism js?



In [ ]:

function testPrism() {
  var left, right;
  var r = importer.interpret(['search notebook component', 'files in project', 'diff code blocks']);
  left = r[0].code;
  var {listInProject} = importer.import('files in project');
  var diffTwoTexts = importer.import('diff code blocks');
  
  var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
  var project = PROFILE_PATH + '/universal';
  
  var components = listInProject(project, '**/search.component.ts');
  right = fs.readFileSync(components[0]).toString();
  return diffTwoTexts(left, right)
}

module.exports = testPrism

